# Packages

In [258]:
# Visualisation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from pyvis.network import Network

# Data analysis / Data processing
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', None)
pd.options.display.float_format = "{:,.2f}".format
from datetime import time, timedelta, datetime
import numpy as np
import networkx as nx
from collections import defaultdict
import ast

# Maths & Stats
import math 
import scipy.stats as st
from scipy import stats
from scipy.stats import norm
import statsmodels.stats.weightstats as ws
from statsmodels.stats.proportion import test_proportions_2indep
import AB_library

# System library
import os
import ipywidgets
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
tqdm.pandas()
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_format='retina'
# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
import openpyxl

# Data connection
from google.cloud import bigquery
bigquery_client = bigquery.Client(project='analytics-dev-333113')


# Useful functions
def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline"'), 
        raw=True
    )

## Functions

In [147]:
def ztest_proportion(
    df: pd.DataFrame,
    metric_col: str,
    ab_group_col: str,
    pairs_list: List[Tuple[int, int]] = [(0, 1)],
    corrected_ci: float = 0.95,
    flag_notation: bool = False
    ) -> pd.DataFrame:
    """Perform proportion tests between two groups."""
    res_table = pd.DataFrame()
    tail = (1 + corrected_ci) / 2
    for pair in pairs_list:
        num0 = df[df[ab_group_col] == pair[0]][metric_col].sum()
        denom0 = df[df[ab_group_col] == pair[0]][metric_col].count()
        num1 = df[df[ab_group_col] == pair[1]][metric_col].sum()
        denom1 = df[df[ab_group_col] == pair[1]][metric_col].count()
        p0 = num0 / denom0
        p1 = num1 / denom1
        std0 = df[df[ab_group_col] == pair[0]][metric_col].std()
        std1 = df[df[ab_group_col] == pair[1]][metric_col].std()
        r = test_proportions_2indep(
            num0, denom0,
            num1, denom1,
            value=0,
            method='wald',
            compare='diff',
            alternative='two-sided',
            return_results = True
        )
        se = np.sqrt(r.variance)
        delta = p1 - p0
        delta_per = (p1 / p0 - 1) * 100
        lb = delta - stats.norm.ppf(tail) * se
        ub = delta + stats.norm.ppf(tail) * se
        lb_per = lb * 100 / p0
        ub_per = ub * 100 / p0
        
        if flag_notation == True:
            print(f'\nComparison between groups: {pair[0]} and {pair[1]}')
            print(f'statistic: {r.statistic}, pvalue: {r.pvalue}')
            print(f'delta = {delta}')
            print(f'delta,% = {delta_per}%')
            print(f'Confidence interval for delta: ({lb}, {ub})')
            print(f'Confidence interval for delta, %: ({lb_per}, {ub_per})')

        result = pd.DataFrame(
            np.array([metric_col, denom0, denom1, pair[0], pair[1], r.statistic, r.pvalue, p0, p1, delta, delta_per, lb, ub, lb_per, ub_per]).reshape(1, -1),
            columns=['metric_name', 
                     'group0_sample_size', 
                     'group1_sample_size', 
                     'group0', 
                     'group1', 
                     'statistic', 
                     'pvalue', 
                     'mean0', 
                     'mean1', 
                     'diff_mean', 
                     'diff_mean_%', 
                     'lower_boundary', 
                     'upper_boundary', 
                     'lower_boundary_%', 
                     'upper_boundary_%',]
        )
        res_table = pd.concat([res_table, result])

        for column in res_table.columns[5:]:
            res_table[column] = res_table[column].astype(float)
        
    return res_table

def ttest(
    df: pd.DataFrame,
    metric_col: str,
    ab_group_col: str,
    pairs_list: List[Tuple[int, int]] = [(0, 1)],
    corrected_ci: float = 0.95,
    flag_notation: bool = False
    ) -> pd.DataFrame:
    """Perform t-tests between two groups."""
    res_table = pd.DataFrame()
    tail = (1 + corrected_ci) / 2
    for pair in pairs_list:
        sample0 = df.loc[df[ab_group_col] == pair[0], metric_col]
        sample1 = df.loc[df[ab_group_col] == pair[1], metric_col]
        m0 = sample0.mean()
        m1 = sample1.mean()
        v0 = sample0.std()**2
        v1 = sample1.std()**2
        n0 = len(sample0)
        n1 = len(sample1)
        t, pvalue, df_ = ws.ttest_ind(
            sample0,
            sample1,
            alternative='two-sided',
            usevar='unequal'
        )
        se = np.sqrt(v0 / n0 + v1 / n1)
        delta = m1 - m0
        delta_per = (m1 / m0 - 1) * 100
        lb = delta - stats.t.ppf(tail, df_) * se
        ub = delta + stats.t.ppf(tail, df_) * se
        lb_per = lb * 100 / m0
        ub_per = ub * 100 / m0
        
        if flag_notation == True:
            print(f'\nComparison between groups: {pair[0]} and {pair[1]}')
            print(f't-statistic: {t}, pvalue: {pvalue}, df: {df_}')
            print(f'delta = {delta}')
            print(f'delta,% = {delta_per}%')
            print(f'Confidence interval for delta: ({lb}, {ub})')
            print(f'Confidence interval for delta, %: ({lb_per}, {ub_per})')

        result = pd.DataFrame(
            np.array([metric_col, n0, n1, pair[0], pair[1], t, 
            # df_, 
            pvalue, m0, m1, delta, delta_per, lb, ub, lb_per, ub_per]).reshape(1, -1),
            columns=['metric_name', 
                     'group0_sample_size', 
                     'group1_sample_size',
                     'group0', 
                     'group1', 
                     'statistic', 
                    #  'df', 
                     'pvalue', 
                     'mean0', 
                     'mean1', 
                     'diff_mean', 
                     'diff_mean_%', 
                     'lower_boundary', 
                     'upper_boundary', 
                     'lower_boundary_%', 
                     'upper_boundary_%']
        )
        res_table = pd.concat([res_table, result])
    
    for column in res_table.columns[5:]:
        res_table[column] = res_table[column].astype(float)

    return res_table

def method_benjamini_hochberg(
    pvalues: np.ndarray,
    alpha: float = 0.05
    ) -> np.ndarray:
    """Apply the Benjamini-Hochberg procedure for multiple hypothesis testing."""
    m = len(pvalues)
    array_alpha = np.arange(1, m + 1) * alpha / m
    sorted_pvalue_indexes = np.argsort(pvalues)
    res = np.zeros(m)
    for idx, pvalue_index in enumerate(sorted_pvalue_indexes):
        pvalue = pvalues[pvalue_index]
        alpha_ = array_alpha[idx]
        if pvalue <= alpha_:
            res[pvalue_index] = 1
        else:
            break
    return res.astype(int)

# Shapiro-Wilk test & Distributions
def check_normality(df, group_column, value_column):
    groups = df[group_column].unique()

    for group in groups:
        group_data = df[df[group_column] == group][value_column].dropna() 
        stat, p = stats.shapiro(group_data)
        print(f'Group {group}: W={stat:.4f}, p-value={p:.4f}')
        if p > 0.05:
            print(f'Group {group}, Metric: {value_column}: Data is normal distributed')
        else:
            print(f'Group {group}, Metric: {value_column}: Data is not normal distributed')

def plot_distribution(df, group_column, value_column):

    groups = df[group_column].unique()
    fig, axes = plt.subplots(2, 2, figsize=(14, 10), gridspec_kw={'height_ratios': [1, 1.5]})

    sns.histplot(data=df, x=value_column, hue=group_column, kde=True, bins=30, alpha=0.4, ax=axes[0, 0])
    axes[0, 0].set_title("Graph + KDE")
    axes[0, 0].set_xlabel(value_column)
    axes[0, 0].set_ylabel("Frequence")

    sns.boxplot(data=df, x=group_column, y=value_column, ax=axes[0, 1])
    axes[0, 1].set_title("Boxplot grouped")
    axes[0, 1].set_xlabel(group_column)
    axes[0, 1].set_ylabel(value_column)

    sns.histplot(df[df[group_column] == groups[0]][value_column], bins=30, kde=True, color='blue', alpha=0.5, ax=axes[1, 0])
    axes[1, 0].set_title(f'Hist for the {groups[0]}')
    axes[1, 0].set_xlabel(value_column)
    axes[1, 0].set_ylabel("frequence")

    sns.histplot(df[df[group_column] == groups[1]][value_column], bins=30, kde=True, color='orange', alpha=0.5, ax=axes[1, 1])
    axes[1, 1].set_title(f'Hist for the {groups[1]}')
    axes[1, 1].set_xlabel(value_column)
    axes[1, 1].set_ylabel("Frequence")

    plt.tight_layout()
    plt.show()

# Levene's & Bartlet's test
def levene(df, indicator, metric):
    w_stats, p_value = st.levene(
        df[df['group_name'] == 0][indicator], 
        df[df['group_name'] == 1][indicator],
                            center=metric)
    
    alpha = 0.05
    
    if p_value > alpha:
        print(f"Variance are from the same population on {metric}")
    else:
        print(f"Variance are from the different population on {metric}")
    
# Cohen's D
def cohens_d(df, metric):
    group1 = df[df['group_name']==1][metric]
    group2 = df[df['group_name']==0][metric]
    mean1, mean2 = np.mean(group1), np.mean(group2)
     
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    n1, n2 = len(group1), len(group2)
    pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
     
    d = (mean1 - mean2) / pooled_std
     
    # if d <= 0.3:
    #     print(f'Small effect: d ≈ 0-0.3 ({d:.3f})')
    # elif 0.31 <= d <= 0.8:
    #     print(f'Medium effect: d ≈ 0.3-0.8 ({d:.3f})')
    # elif 0.81 <= d <= 1:
    #     print(f'Large effect: d ≈ 0.8-1 ({d:.3f})')

    return d

# SRM
def srm(df):
    srm_df = pd.DataFrame()

    for city in df['city_name'].unique():
        
        observed = [
            (df.query(f'group_name == 0 and city_name == "{city}"')['user_id'].count()), 
            (df.query(f'group_name == 1 and city_name == "{city}"')['user_id'].count())
            ]

        total_traffic = sum(observed)

        expected = [total_traffic/2, total_traffic/2]

        chi = st.chisquare(observed, f_exp = expected)

        if chi[1] < 0.01:
            conclusion = "Sample ratio mismatch (SRM) may be present"
        else:
            conclusion = "Sample ratio mismatch (SRM) probably not present"
            print(f"{city}, {chi[1]}")

        
        new_srm_df = pd.DataFrame(
            [[city, observed, total_traffic, expected, round(chi[1], 3), conclusion]], 
            columns=['city_name',  'sample_sizes', 'total_size', 'expected_sizes', 'chi_value', 'conclusion']
            )

        srm_df = pd.concat([srm_df, new_srm_df]).sort_values(['city_name', 'total_size'], ascending=False).reset_index(drop=True)

    return srm_df

# Calcualting the significance by cities
def calcualate_result(df_cr, df_abs):
    df_results = pd.DataFrame()

    for city in df_cr['city_name'].unique():

        absolute_values_keys_result = df_abs[df_abs['city_name']==f'{city}'].copy()

        cr_df = ztest_proportion(df_cr[df_cr['city_name']==f'{city}'], 'has_ride', 'group_name')
        cr_df['metric'] = 'Conversion'
        cr_df['cohen_d'] = cohens_d(df_cr[df_cr['city_name']==f'{city}'], 'has_ride')

        rides_df = ttest(absolute_values_keys_result, 'rides', 'group_name')
        rides_df['metric'] = 'Quantitive'
        rides_df['cohen_d'] = cohens_d(absolute_values_keys_result, 'rides')

        gmv_df = ttest(absolute_values_keys_result, 'gmv', 'group_name')
        gmv_df['metric'] = 'Quantitive'
        gmv_df['cohen_d'] = cohens_d(absolute_values_keys_result, 'gmv')

        orders_df = ttest(absolute_values_keys_result, 'orders', 'group_name')
        orders_df['metric'] = 'Quantitive'
        orders_df['cohen_d'] = cohens_d(absolute_values_keys_result, 'orders')

        df_total = pd.concat([cr_df, rides_df, gmv_df, orders_df])

        df_total['region'] = city
        df_total['segment'] = 'By city'
        df_total['significance'] = (df_total['pvalue']<0.05)*1
        df_total['corrected_pvalue'] = method_benjamini_hochberg(df_total['pvalue'].values)

        df_results = pd.concat([df_results, df_total])

    total_cr_df = ztest_proportion(df_cr, 'has_ride', 'group_name')
    total_cr_df['metric'] = 'Conversion'
    total_cr_df['cohen_d'] = cohens_d(df_cr, 'has_ride')

    total_rides_df = ttest(df_abs, 'rides', 'group_name')
    total_rides_df['metric'] = 'Quantitive'
    total_rides_df['cohen_d'] = cohens_d(df_abs, 'rides')

    total_gmv_df = ttest(df_abs, 'gmv', 'group_name')
    total_gmv_df['metric'] = 'Quantitive'
    total_gmv_df['cohen_d'] = cohens_d(df_abs, 'gmv')

    total_orders_df = ttest(df_abs, 'orders', 'group_name')
    total_orders_df['metric'] = 'Quantitive'
    total_orders_df['cohen_d'] = cohens_d(df_abs, 'orders')


    total_total_df = pd.concat([total_cr_df, total_rides_df, total_gmv_df, total_orders_df])
    total_total_df['region'] = 'All'
    total_total_df['segment'] = 'Total'
    total_total_df['significance'] = (df_total['pvalue']<0.05)*1
    total_total_df['corrected_pvalue'] = method_benjamini_hochberg(df_total['pvalue'].values)

    df_results = pd.concat([df_results, total_total_df])

    df_results

    return df_results

# Design. 24/7 Liveness switching on

In [4]:
# Share of incidents committed by new users

incident_rate_by_newbies = read_bq("""
    WITH incidents AS (SELECT driver_id AS aggressor_id,
                            'driver'  AS mode,
                            redmine_id,
                            incident_level,
                            incident_date,
                            city_id,
                            city_name,
                            country_name,
                    FROM indriver-bi.safety.vw_safety_incidents_detail
                    WHERE incident_date >= '2024-09-01'
                        AND information_status = 'Confirmed'
                        AND aggressor = 'Driver'
                    UNION ALL
                    SELECT pass_id AS aggressor_id,
                            'pass'  AS mode,
                            redmine_id,
                            incident_level,
                            incident_date,
                            city_id,
                            city_name,
                            country_name,
                    FROM indriver-bi.safety.vw_safety_incidents_detail
                    WHERE incident_date >= '2024-09-01'
                        AND information_status = 'Confirmed'
                        AND aggressor = 'Passenger'),
        registered AS (SELECT id, DATE(created) AS registration_date, mode
                    FROM indriver-e6e40.ods_monolith.tbl_user),
        rides AS (SELECT user_type,
                        user_id,
                        metric_date_utc,
                        SUM(rides_count)
                            OVER (PARTITION BY user_type, user_id ORDER BY metric_date_utc ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS rides_count_cumulative
                FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
                WHERE metric_date_utc >= '2024-03-01')
    SELECT t1.aggressor_id,
        t1.mode,
        t1.redmine_id,
        t1.incident_level,
        t1.incident_date,
        t1.city_id,
        t1.city_name,
        t1.country_name,
        t2.registration_date,
        t3.metric_date_utc,
        COALESCE(t3.rides_count_cumulative, 0) AS rides_count_cumulative
    FROM incidents t1
            LEFT JOIN registered t2
                    ON t1.aggressor_id = t2.id
            LEFT JOIN rides t3
                    ON t1.aggressor_id = t3.user_id
                        AND t3.metric_date_utc <= t1.incident_date
    QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.aggressor_id ORDER BY t3.metric_date_utc DESC) = 1
""")

incident_rate_by_newbies.head()

# Incident rate by cities

incident_rate = read_bq("""
    WITH incidents AS (SELECT incident_date,
                            city_id,
                            city_name,
                            country_name,
                            COUNT(redmine_id) AS incidents
                    FROM indriver-bi.safety.vw_safety_incidents_detail
                    WHERE information_status = 'Confirmed'
                        AND aggressor = 'Passenger'
                    GROUP BY 1, 2, 3, 4),
        rides AS (SELECT metric_date_utc,
                        city_id,
                        SUM(rides_count) AS ride_counts
                FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
                WHERE metric_date_utc >= '2024-09-01'
                    AND user_type = 'pass'
                GROUP BY 1, 2)
    SELECT t1.city_id,
        t1.city_name,
        t1.country_name,
        t1.incident_date,
        DATE_TRUNC(t1.incident_date, WEEK)                     AS weekly,
        incidents,
        ride_counts,
        ROUND(SAFE_DIVIDE(incidents, ride_counts) * 100000, 3) AS ir
    FROM incidents t1
            JOIN rides t2
                ON t1.incident_date = t2.metric_date_utc
                    AND t1.city_id = t2.city_id
""")

incident_rate.head()

In [5]:
# Rides by user, Orders by user, GMV in $

rides_orders_gmv = read_bq("""
SELECT user_id,
       t1.country_id,
       t2.country_name,
       t1.city_id,
       t2.city_name,
       SUM(orders_count) / COUNT(user_id) AS orders,
       SUM(rides_count) / COUNT(user_id)  AS rides,
       SUM(gmv_clean_usd)                 AS gmv
FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
         JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
              ON
                  t1.city_id = t2.city_id
WHERE user_type = 'pass'
  AND t1.city_id IN
      (4825, 4155, 4243, 4197, 4255, 4227, 4225, 4242, 5548, 4404, 4234, 5291, 4396, 6587, 5513, 5495, 4230,
       4272, 4518, 4521, 4519, 4758, 4377, 4537, 4534, 4163, 4532, 4375)
  AND metric_date_utc >= '2024-11-01'
GROUP BY 1, 2, 3, 4, 5
""")

rides_orders_gmv = rides_orders_gmv.drop(index = rides_orders_gmv[rides_orders_gmv['orders'] > np.percentile(rides_orders_gmv['orders'],99)].index)

plt.hist(rides_orders_gmv['orders'], bins=30, color='skyblue', edgecolor='black')
 
# Adding labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Basic Histogram')
 
# Display the plot
plt.show()

In [84]:
# CR to ride upon check

def transform(row):
    if  pd.isnull(row['order_timestamp']):
        return 0
    else:
        return 1

cr_to_ride = read_bq("""
WITH liveness AS (SELECT user_id,
                         os_name,
                         event_dt_part,
                         city_id,
                         country_id,
                         city_name,
                         country_name,
                         fulfilled_flow,
                         MIN(IF(name = 'client.verification_flow_result_status.show' AND
                                LOWER(status) = 'approve',
                                client_time,
                                NULL)) AS status_result
                  FROM (SELECT user_id,
                               name,
                               os_name,
                               event_dt_part,
                               TIMESTAMP_MILLIS(client_time)                                           AS client_time,
                               t1.city_id,
                               t2.city_name,
                               t2.country_id,
                               t2.country_name,
                               IF(LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                                      OVER (PARTITION BY user_id, event_dt_part ORDER BY client_time) IS NULL,
                                  JSON_EXTRACT_SCALAR(payload, '$.verification_flow'),
                                  LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                                      OVER (PARTITION BY user_id, event_dt_part ORDER BY client_time)) AS fulfilled_flow,
                               JSON_EXTRACT_SCALAR(payload, '$.status')                                AS status
                        FROM indriver-e6e40.emart.swrve_event t1
                                 JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                      ON
                                          t1.city_id = t2.city_id
                        WHERE 1 = 1
                          AND name IN (
                                       'client.verification_start.show',
                                       'client.verification_flow_result_status.show'
                            )
                          AND event_dt_part >= '2024-11-01'
                          AND t2.country_id IN (25, 12, 13, 43, 24, 11, 22))
                  GROUP BY 1, 2, 3, 4, 5, 6, 7, 8),
     rides AS (SELECT order_uuid,
                      user_id    AS pass_id,
                      driver_id,
                      city_id    AS order_city_id,
                      country_id AS order_country_id,
                      status_order,
                      order_timestamp,
                      at_pickup_dttm,
                      departed_pickup_dttm,
                      at_destination_dttm,
                      departed_destination_dttm,
                      driveraccept_timestamp,
                      driverarrived_timestamp,
                      driverstarttheride_timestamp,
                      driverdone_timestamp,
                      clientdone_timestamp,
                      clientcancel_timestamp,
                      drivercancel_timestamp,
                      user_reg_date,
                      driver_reg_date,
                      stage,
                      created_date_order_part,
                      duration_in_seconds
               FROM indriver-e6e40.imart.incity_detail_new_order
               WHERE created_date_order_part >= '2024-11-01'
                 AND driveraccept_timestamp IS NOT NULL
                 AND (clientcancel_timestamp IS NULL
                   AND drivercancel_timestamp IS NULL))
SELECT t1.user_id,
       t1.os_name,
       t1.city_id,
       t1.country_id,
       t1.city_name,
       t1.country_name,
       t1.status_result,
       t2.order_timestamp,
       status_order
FROM liveness t1
         LEFT JOIN rides t2
                   ON t1.user_id = t2.pass_id
                       AND t2.order_timestamp BETWEEN status_result AND DATE_ADD(status_result, INTERVAL + 1 DAY)
WHERE 1 = 1
  AND (fulfilled_flow IS NULL OR fulfilled_flow = 'liveness')
QUALIFY ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY order_timestamp ASC) = 1
""")

cr_to_ride['has_ride'] = cr_to_ride.apply(transform, axis=1)
# CR to order upon check

cr_to_order = read_bq("""
WITH liveness AS (SELECT user_id,
                         os_name,
                         event_dt_part,
                         city_id,
                         country_id,
                         city_name,
                         country_name,
                         fulfilled_flow,
                         MIN(IF(name = 'client.verification_flow_result_status.show' AND
                                LOWER(status) = 'approve',
                                client_time,
                                NULL)) AS status_result
                  FROM (SELECT user_id,
                               name,
                               os_name,
                               event_dt_part,
                               TIMESTAMP_MILLIS(client_time)                                           AS client_time,
                               t1.city_id,
                               t2.city_name,
                               t2.country_id,
                               t2.country_name,
                               IF(LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                                      OVER (PARTITION BY user_id, event_dt_part ORDER BY client_time) IS NULL,
                                  JSON_EXTRACT_SCALAR(payload, '$.verification_flow'),
                                  LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                                      OVER (PARTITION BY user_id, event_dt_part ORDER BY client_time)) AS fulfilled_flow,
                               JSON_EXTRACT_SCALAR(payload, '$.status')                                AS status
                        FROM indriver-e6e40.emart.swrve_event t1
                                 JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                      ON
                                          t1.city_id = t2.city_id
                        WHERE 1 = 1
                          AND name IN (
                                       'client.verification_start.show',
                                       'client.verification_flow_result_status.show'
                            )
                          AND event_dt_part >= '2024-11-01'
                          AND t2.country_id IN (25, 12, 13, 43, 24, 11, 22))
                  GROUP BY 1, 2, 3, 4, 5, 6, 7, 8),
     rides AS (SELECT order_uuid,
                      user_id    AS pass_id,
                      driver_id,
                      city_id    AS order_city_id,
                      country_id AS order_country_id,
                      status_order,
                      order_timestamp,
                      at_pickup_dttm,
                      departed_pickup_dttm,
                      at_destination_dttm,
                      departed_destination_dttm,
                      driveraccept_timestamp,
                      driverarrived_timestamp,
                      driverstarttheride_timestamp,
                      driverdone_timestamp,
                      clientdone_timestamp,
                      clientcancel_timestamp,
                      drivercancel_timestamp,
                      user_reg_date,
                      driver_reg_date,
                      stage,
                      created_date_order_part,
                      duration_in_seconds
               FROM indriver-e6e40.imart.incity_detail_new_order
               WHERE created_date_order_part >= '2024-11-01')
SELECT t1.user_id,
       t1.os_name,
       t1.city_id,
       t1.country_id,
       t1.city_name,
       t1.country_name,
       t1.status_result,
       t2.order_timestamp,
       status_order
FROM liveness t1
         LEFT JOIN rides t2
                   ON t1.user_id = t2.pass_id
                       AND t2.order_timestamp BETWEEN status_result AND DATE_ADD(status_result, INTERVAL + 1 DAY)
WHERE 1 = 1
  AND (fulfilled_flow IS NULL OR fulfilled_flow = 'liveness')
QUALIFY ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY order_timestamp ASC) = 1
""")

cr_to_order['has_order'] = cr_to_order.apply(transform, axis=1)

In [101]:
# CR to ride from tapping on Find a driver button within 1 day

cr_to_ride_wo_check = read_bq("""
WITH rides AS (SELECT user_id,
                      t1.country_id,
                      t2.country_name,
                      t1.city_id,
                      t2.city_name,
                      metric_date_utc,
                      orders_count,
                      rides_count,
                      IF(rides_count > 0, 1, 0) AS has_ride
               FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                        JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                             ON
                                 t1.city_id = t2.city_id
               WHERE user_type = 'pass'
                 AND metric_date_utc >= '2024-11-01')
SELECT t1.user_id,
       t1.os_name,
       t1.city_id,
       t2.city_name,
       t2.country_id,
       t2.country_name,
       t1.event_dt_part,
       TIMESTAMP_MILLIS(t1.client_time) AS client_time,
       t1.name,
       metric_date_utc,
       orders_count,
       has_ride
FROM indriver-e6e40.emart.swrve_event t1
         JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
              ON
                  t1.city_id = t2.city_id
         LEFT JOIN rides t3
                   ON t1.user_id = t3.user_id
                       AND t3.metric_date_utc BETWEEN event_dt_part AND DATE_ADD(event_dt_part, INTERVAL + 1 DAY)
WHERE event_dt_part >= '2024-11-01'
  AND name = 'city.client.create_order.click'
  AND t1.city_id IN
      (4825, 4155, 4243, 4197, 4255, 4227, 4225, 4242, 5548, 4404, 4234, 5291, 4396, 6587, 5513, 5495, 4230, 4272,
       4518,
       4521, 4519, 4758, 4377, 4537, 4534, 4163, 4532, 4375)
QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.user_id ORDER BY client_time) = 1
""")

cr_to_ride_wo_check = cr_to_ride_wo_check[~cr_to_ride_wo_check['has_ride'].isna()]

cr_to_ride_wo_check = cr_to_ride_wo_check[(~cr_to_ride_wo_check['os_name'].isna()) & (cr_to_ride_wo_check['os_name'] != 'ipados') & (~cr_to_ride_wo_check['has_ride'].isna())]



In [ ]:
# Sample size

sample_size = cycle_sql('2024-12-01', '2024-12-7', """
WITH rides_before AS (SELECT DISTINCT user_id
                      FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
                      WHERE user_type = 'pass'
                        AND city_id IN
                            (4825, 4155, 4243, 4197, 4255, 4227, 4225, 4242, 5548, 4404, 4234, 5291, 4396, 6587, 5513,
                             5495, 4230, 4272, 4518,
                             4521, 4519, 4758, 4377, 4537, 4534, 4163, 4532, 4375)
                        AND rides_count > 0
                        AND metric_date_utc BETWEEN '2024-01-01' AND '{date}')
SELECT t1.city_id,
       t2.city_name,
       t2.country_id,
       t2.country_name,
       event_dt_part AS daily,
       COUNT(DISTINCT user_id) AS users
FROM indriver-e6e40.emart.swrve_event t1
         JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
              ON
                  t1.city_id = t2.city_id
WHERE event_dt_part = '{date}'
  AND user_id NOT IN (SELECT user_id FROM rides_before)
  AND name = 'city.client.create_order.click'
  AND t1.city_id IN
      (4825, 4155, 4243, 4197, 4255, 4227, 4225, 4242, 5548, 4404, 4234, 5291, 4396, 6587, 5513, 5495, 4230, 4272, 4518,
       4521, 4519, 4758, 4377, 4537, 4534, 4163, 4532, 4375)
GROUP BY 1, 2, 3, 4, 5
""")


In [87]:
# Preparing how many users actually hit the check

daily = sample_size.groupby(['daily'], as_index=False)['users'].sum()['users'].mean().round(0)
weekly = sample_size.groupby(['daily'], as_index=False)['users'].sum()['users'].sum().round(0)

In [285]:
effects = [1.01, 1.015, 1.05, 1.1]  # MDE in percents
sizes = [int(daily), int(weekly), int(weekly)*2, int(daily)*30]  # Size of each group
first_type_errors = [0.01, 0.05]
second_type_errors = [0.1, 0.2]

liveness_where_no_checks = [6587, 4230, 5495, 5513, 4272, 4396]
liveness_where_at_nighttime = [4155, 4825, 5291, 4234, 4404, 5548, 4143, 4198, 4225, 4227, 4255, 4197, 4243]
liveness_where_id_reg_step = [5483, 5483, 4377, 4532, 4521, 4537, 4758, 4163, 4534, 4519]
liveness_where_id_reg_step_and_nighttime = [4375]

total_cities = [(6587, 4230, 5495, 5513, 4272, 4396), (4155, 4825, 5291, 4234, 4404, 5548, 4143, 4198, 4225, 4227, 4255, 4197, 4243), (5483, 5483, 4377, 4532, 4521, 4537, 4758, 4163, 4534, 4519), (4375, 0)]

features = ['power', 'size', 'effect']

power = pd.DataFrame()
size = pd.DataFrame()
effect = pd.DataFrame()

for id, city in enumerate(total_cities):

    if id == 0 and city[0] in liveness_where_no_checks:
            segment = '24/7 Liveness w/o checks at all'
    elif id == 1 and city[0] in liveness_where_at_nighttime:
            segment = '24/7 Liveness w checks at night time'
    elif id == 2 and city[0] in liveness_where_id_reg_step:
            segment = '24/7 Liveness w ID checks in reg'
    elif id == 3 and city[0] in liveness_where_id_reg_step_and_nighttime:
            segment = '24/7 Liveness w ID checks in reg & Nightime'


    designer_orders = Designer(
        dataframe=rides_orders_gmv[rides_orders_gmv['city_id'].isin(city)], 
        metrics='orders'
        )

    designer_rides = Designer(
        dataframe=rides_orders_gmv[rides_orders_gmv['city_id'].isin(city)], 
        metrics='rides'
        )

    designer_gmv = Designer(
        dataframe=rides_orders_gmv[rides_orders_gmv['city_id'].isin(city)], 
        metrics='gmv'
        )

    designer_cr_to_orders = Designer(
        dataframe=cr_to_order[cr_to_order['city_id'].isin(city)], 
        metrics='has_order'
        )

    designer_cr_to_rides = Designer(
        dataframe=cr_to_ride[cr_to_ride['city_id'].isin(city)], 
        metrics='has_ride'
        )

    designer_cr_to_rides_wo_check_OEC = Designer(
        dataframe=cr_to_ride_wo_check[cr_to_ride_wo_check['city_id'].isin(city)], 
        metrics='has_ride'
        )

    designer_orders.set_first_errors(first_type_errors)
    designer_orders.set_second_errors(second_type_errors)

    designer_rides.set_first_errors(first_type_errors)
    designer_rides.set_second_errors(second_type_errors)

    designer_gmv.set_first_errors(first_type_errors)
    designer_gmv.set_second_errors(second_type_errors)

    designer_cr_to_orders.set_first_errors(first_type_errors)
    designer_cr_to_orders.set_second_errors(second_type_errors)

    designer_cr_to_rides.set_first_errors(first_type_errors)
    designer_cr_to_rides.set_second_errors(second_type_errors)

    designer_cr_to_rides_wo_check_OEC.set_first_errors(first_type_errors)
    designer_cr_to_rides_wo_check_OEC.set_second_errors(second_type_errors)

    for feature in features:

        df_orders = designer_orders.run(
            to_design=f'{feature}', 
            method='theory', 
            effects=effects,
            sizes=sizes
            )
        df_orders['Metric'] = 'avg_orders_by_user'
        df_orders['Segment'] = f'{segment}'

        df_rides = designer_rides.run(
            to_design=f'{feature}', 
            method='theory', 
            effects=effects,
            sizes=sizes
            )
        df_rides['Metric'] = 'avg_rides_by_user'
        df_rides['Segment'] = f'{segment}'

        df_gmv = designer_gmv.run(
            to_design=f'{feature}', 
            method='theory', 
            effects=effects,
            sizes=sizes
            )
        df_gmv['Metric'] = 'gmv_in_dollars'
        df_gmv['Segment'] = f'{segment}'

        df_cr_to_orders = designer_cr_to_orders.run(
            to_design=f'{feature}', 
            method='theory', 
            effects=effects,
            sizes=sizes
            )
        df_cr_to_orders['Metric'] = 'cr_to_orders'
        df_cr_to_orders['Segment'] = f'{segment}'

        df_cr_to_rides = designer_cr_to_rides.run(
            to_design=f'{feature}', 
            method='theory', 
            effects=effects,
            sizes=sizes
            )
        df_cr_to_rides['Metric'] = 'cr_to_rides'
        df_cr_to_rides['Segment'] = f'{segment}'

        df_cr_to_rides_wo_check_OEC = designer_cr_to_rides_wo_check_OEC.run(
            to_design=f'{feature}', 
            method='theory', 
            effects=effects,
            sizes=sizes
            )
        df_cr_to_rides_wo_check_OEC['Metric'] = 'cr_to_rides_wo_check'
        df_cr_to_rides_wo_check_OEC['Segment'] = f'{segment}'

        if feature == 'power':
            power = pd.concat([power, df_orders, df_rides, df_gmv, df_cr_to_orders, df_cr_to_rides, df_cr_to_rides_wo_check_OEC])
        elif feature == 'size':
            size = pd.concat([size, df_orders, df_rides, df_gmv, df_cr_to_orders, df_cr_to_rides, df_cr_to_rides_wo_check_OEC])
        else:
            effect = pd.concat([effect, df_orders, df_rides, df_gmv, df_cr_to_orders, df_cr_to_rides, df_cr_to_rides_wo_check_OEC])


In [ ]:
# need to add segments for each cities (Which test it is)

for city in cr_to_ride_wo_check['city_id'].unique():

        if city in liveness_where_no_checks:
                segment = '24/7 Liveness w/o checks at all'
        elif city in liveness_where_at_nighttime:
                segment = '24/7 Liveness w checks at night time'
        elif city in liveness_where_id_reg_step:
                segment = '24/7 Liveness w ID checks in reg'
        elif city in liveness_where_id_reg_step_and_nighttime:
                segment = '24/7 Liveness w ID checks in reg & Nightime'

        designer_cr_to_rides_OEC_by_cities = Designer(
        dataframe=cr_to_ride_wo_check.query(f"city_id == {city}"), 
        metrics='has_ride'
        )

        designer_cr_to_rides.set_first_errors(first_type_errors)
        designer_cr_to_rides.set_second_errors(second_type_errors)

        df_cr_to_rides_OEC_by_cities = designer_cr_to_rides_OEC_by_cities.run(
        to_design='effect', 
        method='theory', 
        effects=effects,
        sizes=sizes
        )

        city_name = cr_to_ride_wo_check.query(f"city_id == '{city}'")['city_name'].unique(),

        df_cr_to_rides_OEC_by_cities['Metric'] = 'cr_to_rides_wo_check'
        df_cr_to_rides_OEC_by_cities['Segment'] = f'{segment}'
        df_cr_to_rides_OEC_by_cities['City'] = f'{city}'

        design_by_cities = pd.concat([design_by_cities, df_cr_to_rides_OEC_by_cities])

design_by_cities

# Results. The key metric for calculating: Rides, Orders, CR to ride, GMV

##### 1) Cohen's D
$d = \frac{\bar{X}_1 - \bar{X}_2}{s}$;
$s = \sqrt{\frac{(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2}{n_1 + n_2 - 2}}$

##### 2) Shapiro-Wilk test & Distributions
##### 3) Levene's & Bartlet's test
##### 4) Z/T-test

## Summarizing - Liveness In cities where nothing works 

#### Data pulling

In [222]:
# Rides, Orders, GMV

absolute_values_keys = read_bq("""
    SELECT t1.user_id,
       t3.group_name,
       t3.group_id,
       t1.country_id,
       t2.country_name,
       t1.city_id,
       t2.city_name,
       SUM(orders_count) / COUNT(t1.user_id) AS avg_orders,
       SUM(rides_count) / COUNT(t1.user_id)  AS avg_rides,
       SUM(rides_count)                      AS rides,
       SUM(orders_count)                     AS orders,
       SUM(gmv_clean_usd)                    AS gmv
    FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                ON
                    t1.city_id = t2.city_id
            JOIN (SELECT user_id,
                        group_id,
                        IF(group_id = 4528379, 1, 0) AS group_name, -- Change the group
                        city_id
                FROM indriver-e6e40.ss_ab_platform_mart.markup_users
                WHERE test_id = 2435) t3 -- Change the test ID
                ON t1.user_id = t3.user_id
    WHERE user_type = 'pass'
    AND metric_date_utc >= '2025-01-20'
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    """)


# CR to ride tapping on Find a driver button (Total CR to ride)

total_cr_to_ride = read_bq("""
    WITH rides AS (SELECT user_id,
                        t1.country_id,
                        t2.country_name,
                        t1.city_id,
                        t2.city_name,
                        metric_date_utc,
                        SUM(orders_count) AS orders_count,
                        SUM(rides_count)  AS rides_count,
                FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                ON
                                    t1.city_id = t2.city_id
                WHERE user_type = 'pass'
                    AND metric_date_utc >= '2025-01-20'
                GROUP BY 1, 2, 3, 4, 5, 6),
        total_ds AS (SELECT t1.user_id,
                            t1.city_id,
                            t2.city_name,
                            t2.country_name,
                            test_id,
                            group_id,
                            IF(group_id = 4528379, 1, 0) AS group_name,
                            created_dt_part,
                            IF(t3.rides_count > 0, 1, 0)                   AS has_ride
                    FROM indriver-e6e40.ss_ab_platform_mart.markup_users t1
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                    ON
                                        t1.city_id = t2.city_id
                            LEFT JOIN rides t3
                                        ON t1.user_id = t3.user_id
                                            AND
                                            metric_date_utc BETWEEN created_dt_part AND DATE_ADD(created_dt_part, INTERVAL + 1 DAY)
                    WHERE test_id = 2435)
    SELECT user_id,
        city_id,
        group_id,
        city_name,
        country_name,
        test_id,
        group_name,
        MAX(has_ride) AS has_ride
    FROM total_ds
    GROUP BY 1, 2, 3, 4, 5, 6, 7
""")

total_cr_to_ride.head()

,user_id,city_id,group_id,city_name,country_name,test_id,group_name,has_ride
0,168177619,5513,4528378,San Jose,Costa Rica,2435,0,0
1,203500773,5513,4528379,San Jose,Costa Rica,2435,1,0
2,25547617,5513,4528378,San Jose,Costa Rica,2435,0,0
3,32026949,5513,4528379,San Jose,Costa Rica,2435,1,1
4,29673673,5513,4528379,San Jose,Costa Rica,2435,1,1


In [224]:
absolute_values_keys[absolute_values_keys['city_id'].isin([6587, 4230, 5495, 5513, 4272, 4396])].groupby(['city_name', 'group_name'], as_index=False)[['rides', 'orders', 'gmv']].sum()\
                .sort_values(['city_name', 'group_name'], ascending=False)


,city_name,group_name,rides,orders,gmv
9,San Jose,1,322579,615394,"1,581,435.53"
8,San Jose,0,317512,608638,"1,563,749.66"
7,Mendoza,1,1,193,3.83
6,Mendoza,0,3,180,14.71
5,Manta,1,87487,129947,"194,836.19"
4,Manta,0,88121,131420,"196,623.10"
3,Chihuahua,1,93508,180254,"343,885.07"
2,Chihuahua,0,93764,181371,"344,949.83"
1,Chiclayo,1,211452,322107,"522,322.75"
0,Chiclayo,0,211561,322953,"521,694.71"


#### Effect calculating (Applying criteria and using charts)


In [244]:
df = calcualate_result(total_cr_to_ride, absolute_values_keys)

In [245]:
df[['metric_name', 'group0_sample_size', 'group1_sample_size', 'pvalue', 'mean0', 'mean1', 'diff_mean',
       'diff_mean_%', 'lower_boundary', 'upper_boundary', 'lower_boundary_%',
       'upper_boundary_%', 'metric', 'cohen_d', 'region', 'segment']]

,metric_name,group0_sample_size,group1_sample_size,pvalue,mean0,mean1,diff_mean,diff_mean_%,lower_boundary,upper_boundary,lower_boundary_%,upper_boundary_%,metric,cohen_d,region,segment
0,has_ride,13913,13738,0.85,0.34,0.34,0.00,0.33,-0.01,0.01,-2.97,3.62,Conversion,0.00,Serra,By city
0,rides,9960,9917,0.48,2.99,3.04,0.05,1.61,-0.09,0.18,-2.88,6.10,Quantitive,0.01,Serra,By city
0,gmv,9960,9917,0.44,11.92,12.13,0.21,1.76,-0.33,0.75,-2.75,6.28,Quantitive,0.01,Serra,By city
0,orders,9960,9917,0.37,8.82,8.98,0.16,1.83,-0.19,0.51,-2.18,5.84,Quantitive,0.01,Serra,By city
0,has_ride,14580,14786,0.96,0.24,0.24,0.00,0.11,-0.01,0.01,-3.93,4.16,Conversion,0.00,Florianópolis,By city
0,rides,8003,8147,0.34,2.60,2.66,0.07,2.57,-0.07,0.20,-2.69,7.83,Quantitive,0.02,Florianópolis,By city
0,gmv,8003,8147,0.23,10.98,11.34,0.37,3.33,-0.23,0.96,-2.06,8.73,Quantitive,0.02,Florianópolis,By city
0,orders,8003,8147,0.24,8.49,8.72,0.23,2.68,-0.15,0.61,-1.81,7.16,Quantitive,0.02,Florianópolis,By city
0,has_ride,4940,4878,0.05,0.25,0.23,-0.02,-6.72,-0.03,0.00,-13.57,0.12,Conversion,-0.04,Aracaju,By city
0,rides,2684,2560,0.66,2.74,2.80,0.06,2.07,-0.20,0.31,-7.12,11.25,Quantitive,0.01,Aracaju,By city


## Summarizing - Liveness In cities where liveness works at night time

#### Data pulling

In [184]:
# Rides, Orders, GMV

absolute_values_keys = read_bq("""
    SELECT t1.user_id,
       t3.group_name,
       t3.group_id,
       t1.country_id,
       t2.country_name,
       t1.city_id,
       t2.city_name,
       SUM(orders_count) / COUNT(t1.user_id) AS avg_orders,
       SUM(rides_count) / COUNT(t1.user_id)  AS avg_rides,
       SUM(rides_count)                      AS rides,
       SUM(orders_count)                     AS orders,
       SUM(gmv_clean_usd)                    AS gmv
    FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                ON
                    t1.city_id = t2.city_id
            JOIN (SELECT user_id,
                        group_id,
                        IF(group_id = 4528385, 1, 0) AS group_name, -- Change the group
                        city_id
                FROM indriver-e6e40.ss_ab_platform_mart.markup_users
                WHERE test_id = 2438) t3 -- Change the test ID
                ON t1.user_id = t3.user_id
    WHERE user_type = 'pass'
    AND metric_date_utc >= '2025-01-20'
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    """)


# CR to ride tapping on Find a driver button (Total CR to ride)

total_cr_to_ride = read_bq("""
    WITH rides AS (SELECT user_id,
                        t1.country_id,
                        t2.country_name,
                        t1.city_id,
                        t2.city_name,
                        metric_date_utc,
                        SUM(orders_count) AS orders_count,
                        SUM(rides_count)  AS rides_count,
                FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                ON
                                    t1.city_id = t2.city_id
                WHERE user_type = 'pass'
                    AND metric_date_utc >= '2025-01-20'
                GROUP BY 1, 2, 3, 4, 5, 6),
        total_ds AS (SELECT t1.user_id,
                            t1.city_id,
                            t2.city_name,
                            t2.country_name,
                            test_id,
                            group_id,
                            IF(group_id = 4528385, 1, 0) AS group_name,
                            created_dt_part,
                            IF(t3.rides_count > 0, 1, 0)                   AS has_ride
                    FROM indriver-e6e40.ss_ab_platform_mart.markup_users t1
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                    ON
                                        t1.city_id = t2.city_id
                            LEFT JOIN rides t3
                                        ON t1.user_id = t3.user_id
                                            AND
                                            metric_date_utc BETWEEN created_dt_part AND DATE_ADD(created_dt_part, INTERVAL + 1 DAY)
                    WHERE test_id = 2438)
    SELECT user_id,
        city_id,
        group_id,
        city_name,
        country_name,
        test_id,
        group_name,
        MAX(has_ride) AS has_ride
    FROM total_ds
    GROUP BY 1, 2, 3, 4, 5, 6, 7
""")

total_cr_to_ride.head()

,user_id,city_id,group_id,city_name,country_name,test_id,group_name,has_ride
0,131158799,5291,4528385,Toluca,Mexico,2438,1,0
1,273468316,5291,4528385,Toluca,Mexico,2438,1,0
2,271808959,5291,4528385,Toluca,Mexico,2438,1,0
3,225489344,5291,4528384,Toluca,Mexico,2438,0,0
4,17114101,5291,4528384,Toluca,Mexico,2438,0,0


In [221]:
absolute_values_keys[absolute_values_keys['city_id'].isin([4155, 4825, 5291, 4234, 4404, 5548, 4143, 4198, 4225, 4227, 4255, 4197, 4243])].groupby(['city_name', 'group_name'], as_index=False)[['rides', 'orders', 'gmv']].sum()\
                .sort_values(['city_name', 'group_name'], ascending=False)


,city_name,group_name,rides,orders,gmv
25,Toluca,1,13791,31427,"47,346.58"
24,Toluca,0,13332,30698,"45,731.57"
23,Santo Domingo,1,831788,1327413,"2,721,440.26"
22,Santo Domingo,0,841988,1339614,"2,753,726.61"
21,Santiago de Queretaro,1,367852,635573,"1,442,762.47"
20,Santiago de Queretaro,0,365739,633897,"1,436,240.69"
19,Quito,1,619686,1093670,"1,946,127.25"
18,Quito,0,618696,1092460,"1,947,579.97"
17,Panama,1,525598,789479,"1,976,538.84"
16,Panama,0,519819,780542,"1,957,931.83"


#### Effect calculating (Applying criteria and using charts)

In [191]:
df = calcualate_result(total_cr_to_ride, absolute_values_keys)

df[['metric_name', 'group0_sample_size', 'group1_sample_size', 'pvalue', 'mean0', 'mean1', 'diff_mean',
       'diff_mean_%', 'lower_boundary', 'upper_boundary', 'lower_boundary_%',
       'upper_boundary_%', 'metric', 'cohen_d', 'region', 'segment']]

,metric_name,group0_sample_size,group1_sample_size,pvalue,mean0,mean1,diff_mean,diff_mean_%,lower_boundary,upper_boundary,lower_boundary_%,upper_boundary_%,metric,cohen_d,region,segment
0,has_ride,8330,8276,0.21,0.30,0.29,-0.01,-2.94,-0.02,0.01,-7.54,1.66,Conversion,-0.02,Toluca,By city
0,has_ride,217407,216374,0.48,0.36,0.36,-0.00,-0.28,-0.00,0.00,-1.08,0.51,Conversion,-0.00,Mexico city,By city
0,has_ride,39551,39008,0.27,0.27,0.27,0.00,1.32,-0.00,0.01,-1.00,3.63,Conversion,0.01,Monterrey,By city
0,has_ride,479763,480932,0.50,0.42,0.42,-0.00,-0.16,-0.00,0.00,-0.63,0.31,Conversion,-0.00,Bogota,By city
0,has_ride,191014,190244,0.27,0.48,0.49,0.00,0.37,-0.00,0.00,-0.28,1.03,Conversion,0.00,Guatemala City,By city
0,has_ride,95356,95544,0.81,0.37,0.37,0.00,0.14,-0.00,0.00,-1.03,1.32,Conversion,0.00,Guadalajara,By city
0,has_ride,88614,88501,0.73,0.51,0.51,-0.00,-0.16,-0.01,0.00,-1.07,0.75,Conversion,-0.00,Ciudad Juarez,By city
0,has_ride,89173,89000,0.50,0.54,0.54,0.00,0.29,-0.00,0.01,-0.56,1.15,Conversion,0.00,Santiago de Queretaro,By city
0,has_ride,243024,242042,0.22,0.57,0.57,-0.00,-0.31,-0.00,0.00,-0.80,0.18,Conversion,-0.00,Barranquilla,By city
0,has_ride,95265,94914,0.80,0.39,0.39,0.00,0.15,-0.00,0.00,-0.98,1.28,Conversion,0.00,Cali,By city


## Summarizing - Liveness In cities where ID checking in registration step + Liveness disabled

#### Data pulling

In [238]:
# Rides, Orders, GMV

absolute_values_keys = read_bq("""
    SELECT t1.user_id,
       t3.group_name,
       t3.group_id,
       t1.country_id,
       t2.country_name,
       t1.city_id,
       t2.city_name,
       SUM(orders_count) / COUNT(t1.user_id) AS avg_orders,
       SUM(rides_count) / COUNT(t1.user_id)  AS avg_rides,
       SUM(rides_count)                      AS rides,
       SUM(orders_count)                     AS orders,
       SUM(gmv_clean_usd)                    AS gmv
    FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                ON
                    t1.city_id = t2.city_id
            JOIN (SELECT user_id,
                        group_id,
                        IF(group_id = 4528383, 1, 0) AS group_name, -- Change the group
                        city_id
                FROM indriver-e6e40.ss_ab_platform_mart.markup_users
                WHERE test_id = 2437) t3 -- Change the test ID
                ON t1.user_id = t3.user_id
    WHERE user_type = 'pass'
    AND metric_date_utc >= '2025-01-20'
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    """)


# CR to ride tapping on Find a driver button (Total CR to ride)

total_cr_to_ride = read_bq("""
    WITH rides AS (SELECT user_id,
                        t1.country_id,
                        t2.country_name,
                        t1.city_id,
                        t2.city_name,
                        metric_date_utc,
                        SUM(orders_count) AS orders_count,
                        SUM(rides_count)  AS rides_count,
                FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                ON
                                    t1.city_id = t2.city_id
                WHERE user_type = 'pass'
                    AND metric_date_utc >= '2025-01-20'
                GROUP BY 1, 2, 3, 4, 5, 6),
        total_ds AS (SELECT t1.user_id,
                            t1.city_id,
                            t2.city_name,
                            t2.country_name,
                            test_id,
                            group_id,
                            IF(group_id = 4528383, 1, 0) AS group_name,
                            created_dt_part,
                            IF(t3.rides_count > 0, 1, 0)                   AS has_ride
                    FROM indriver-e6e40.ss_ab_platform_mart.markup_users t1
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                    ON
                                        t1.city_id = t2.city_id
                            LEFT JOIN rides t3
                                        ON t1.user_id = t3.user_id
                                            AND
                                            metric_date_utc BETWEEN created_dt_part AND DATE_ADD(created_dt_part, INTERVAL + 1 DAY)
                    WHERE test_id = 2437)
    SELECT user_id,
        city_id,
        group_id,
        city_name,
        country_name,
        test_id,
        group_name,
        MAX(has_ride) AS has_ride
    FROM total_ds
    GROUP BY 1, 2, 3, 4, 5, 6, 7
""")

total_cr_to_ride.head()

,user_id,city_id,group_id,city_name,country_name,test_id,group_name,has_ride
0,265319923,4163,4528383,Serra,Brazil,2437,1,0
1,209535055,4163,4528383,Serra,Brazil,2437,1,0
2,234140306,4163,4528383,Serra,Brazil,2437,1,0
3,151875433,4163,4528382,Serra,Brazil,2437,0,0
4,145226124,4163,4528383,Serra,Brazil,2437,1,0


In [228]:
absolute_values_keys[absolute_values_keys['city_id'].isin([5483, 4518, 4377, 4532, 4521, 4537, 4758, 4163, 4534, 4519])].groupby(['city_name', 'group_name'], as_index=False)[['rides', 'orders', 'gmv']].sum()\
                .sort_values(['city_name', 'group_name'], ascending=False)


,city_name,group_name,rides,orders,gmv
19,Vitoria,1,35513,97086,"148,223.19"
18,Vitoria,0,35868,96682,"149,458.42"
17,São Luís,1,7066,25312,"23,578.69"
16,São Luís,0,7288,26126,"24,704.01"
15,Serra,1,30110,89091,"120,285.70"
14,Serra,0,29761,87873,"118,715.53"
13,Santos,1,806,3028,"3,668.02"
12,Santos,0,814,3396,"3,742.76"
11,Rondonópolis,1,71868,102201,"186,852.35"
10,Rondonópolis,0,71279,101698,"186,512.45"


#### Effect calculating (Applying criteria and using charts)


In [241]:
df = calcualate_result(total_cr_to_ride, absolute_values_keys)

df[['metric_name', 'group0_sample_size', 'group1_sample_size', 'pvalue', 'mean0', 'mean1', 'diff_mean',
       'diff_mean_%', 'lower_boundary', 'upper_boundary', 'lower_boundary_%',
       'upper_boundary_%', 'metric', 'cohen_d', 'region', 'segment']]

,metric_name,group0_sample_size,group1_sample_size,pvalue,mean0,mean1,diff_mean,diff_mean_%,lower_boundary,upper_boundary,lower_boundary_%,upper_boundary_%,metric,cohen_d,region,segment
0,has_ride,13913,13738,0.85,0.34,0.34,0.00,0.33,-0.01,0.01,-2.97,3.62,Conversion,0.00,Serra,By city
0,has_ride,14580,14786,0.96,0.24,0.24,0.00,0.11,-0.01,0.01,-3.93,4.16,Conversion,0.00,Florianópolis,By city
0,has_ride,4940,4878,0.05,0.25,0.23,-0.02,-6.72,-0.03,0.00,-13.57,0.12,Conversion,-0.04,Aracaju,By city
0,has_ride,19456,19406,0.52,0.30,0.29,-0.00,-1.01,-0.01,0.01,-4.07,2.05,Conversion,-0.01,Vitoria,By city
0,has_ride,4363,4341,0.18,0.20,0.19,-0.01,-5.77,-0.03,0.01,-14.14,2.59,Conversion,-0.03,Maceió,By city
0,has_ride,42751,42880,0.33,0.30,0.30,0.00,1.02,-0.00,0.01,-1.03,3.06,Conversion,0.01,Goiânia,By city
0,has_ride,6134,6009,0.34,0.21,0.22,0.01,3.33,-0.01,0.02,-3.55,10.21,Conversion,0.02,São Luís,By city
0,has_ride,17212,17262,0.36,0.60,0.60,-0.00,-0.81,-0.02,0.01,-2.53,0.91,Conversion,-0.01,Rondonópolis,By city
0,has_ride,3484,3539,0.44,0.11,0.11,-0.01,-5.12,-0.02,0.01,-18.20,7.97,Conversion,-0.02,Santos,By city
0,has_ride,45676,46047,0.91,0.23,0.23,0.00,0.14,-0.01,0.01,-2.20,2.49,Conversion,0.00,Buenos Aires,By city


## Summarizing - Liveness In cities where ID checking in registration step + Night time liveness

#### Data pulling

In [231]:
# Rides, Orders, GMV

absolute_values_keys = read_bq("""
    SELECT t1.user_id,
       t3.group_name,
       t3.group_id,
       t1.country_id,
       t2.country_name,
       t1.city_id,
       t2.city_name,
       SUM(orders_count) / COUNT(t1.user_id) AS avg_orders,
       SUM(rides_count) / COUNT(t1.user_id)  AS avg_rides,
       SUM(rides_count)                      AS rides,
       SUM(orders_count)                     AS orders,
       SUM(gmv_clean_usd)                    AS gmv
    FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                ON
                    t1.city_id = t2.city_id
            JOIN (SELECT user_id,
                        group_id,
                        IF(group_id = 4528381, 1, 0) AS group_name, -- Change the group
                        city_id
                FROM indriver-e6e40.ss_ab_platform_mart.markup_users
                WHERE test_id = 2436) t3 -- Change the test ID
                ON t1.user_id = t3.user_id
    WHERE user_type = 'pass'
    AND metric_date_utc >= '2025-01-20'
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    """)


# CR to ride tapping on Find a driver button (Total CR to ride)

total_cr_to_ride = read_bq("""
    WITH rides AS (SELECT user_id,
                        t1.country_id,
                        t2.country_name,
                        t1.city_id,
                        t2.city_name,
                        metric_date_utc,
                        SUM(orders_count) AS orders_count,
                        SUM(rides_count)  AS rides_count,
                FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                ON
                                    t1.city_id = t2.city_id
                WHERE user_type = 'pass'
                    AND metric_date_utc >= '2025-01-20'
                GROUP BY 1, 2, 3, 4, 5, 6),
        total_ds AS (SELECT t1.user_id,
                            t1.city_id,
                            t2.city_name,
                            t2.country_name,
                            test_id,
                            group_id,
                            IF(group_id = 4528381, 1, 0) AS group_name,
                            created_dt_part,
                            IF(t3.rides_count > 0, 1, 0)                   AS has_ride
                    FROM indriver-e6e40.ss_ab_platform_mart.markup_users t1
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                    ON
                                        t1.city_id = t2.city_id
                            LEFT JOIN rides t3
                                        ON t1.user_id = t3.user_id
                                            AND
                                            metric_date_utc BETWEEN created_dt_part AND DATE_ADD(created_dt_part, INTERVAL + 1 DAY)
                    WHERE test_id = 2436)
    SELECT user_id,
        city_id,
        group_id,
        city_name,
        country_name,
        test_id,
        group_name,
        MAX(has_ride) AS has_ride
    FROM total_ds
    GROUP BY 1, 2, 3, 4, 5, 6, 7
""")

total_cr_to_ride.head()

,user_id,city_id,group_id,city_name,country_name,test_id,group_name,has_ride
0,164279041,4375,4528380,Rio de Janeiro,Brazil,2436,0,0
1,277539082,4375,4528380,Rio de Janeiro,Brazil,2436,0,0
2,81700079,4375,4528380,Rio de Janeiro,Brazil,2436,0,1
3,61912754,4375,4528380,Rio de Janeiro,Brazil,2436,0,0
4,20286261,4375,4528380,Rio de Janeiro,Brazil,2436,0,0


In [232]:
absolute_values_keys[absolute_values_keys['city_id'].isin([4375])].groupby(['city_name', 'group_name'], as_index=False)[['rides', 'orders', 'gmv']].sum()\
                .sort_values(['city_name', 'group_name'], ascending=False)


,city_name,group_name,rides,orders,gmv
1,Rio de Janeiro,1,64433,236868,"335,431.82"
0,Rio de Janeiro,0,64303,233543,"338,253.05"


#### Effect calculating (Applying criteria and using charts)


In [234]:
df = calcualate_result(total_cr_to_ride, absolute_values_keys)

df[['metric_name', 'group0_sample_size', 'group1_sample_size', 'pvalue', 'mean0', 'mean1', 'diff_mean',
       'diff_mean_%', 'lower_boundary', 'upper_boundary', 'lower_boundary_%',
       'upper_boundary_%', 'metric', 'cohen_d', 'region', 'segment']]

,metric_name,group0_sample_size,group1_sample_size,pvalue,mean0,mean1,diff_mean,diff_mean_%,lower_boundary,upper_boundary,lower_boundary_%,upper_boundary_%,metric,cohen_d,region,segment
0,has_ride,71284,71281,0.35,0.18,0.18,-0.00,-1.04,-0.01,0.00,-3.22,1.15,Conversion,-0.00,Rio de Janeiro,By city
0,rides,36645,36645,0.88,1.75,1.76,0.00,0.20,-0.04,0.05,-2.42,2.82,Quantitive,0.00,Rio de Janeiro,By city
0,gmv,36645,36645,0.54,9.23,9.15,-0.08,-0.83,-0.33,0.17,-3.53,1.86,Quantitive,-0.00,Rio de Janeiro,By city
0,orders,36645,36645,0.17,6.37,6.46,0.09,1.42,-0.04,0.22,-0.61,3.45,Quantitive,0.01,Rio de Janeiro,By city
0,has_ride,71284,71281,0.35,0.18,0.18,-0.00,-1.04,-0.01,0.00,-3.22,1.15,Conversion,-0.00,All,Total
0,rides,41508,41504,0.82,1.82,1.83,0.01,0.28,-0.04,0.05,-2.19,2.76,Quantitive,0.00,All,Total
0,gmv,41508,41504,0.51,9.35,9.27,-0.08,-0.84,-0.31,0.16,-3.35,1.66,Quantitive,-0.00,All,Total
0,orders,41508,41504,0.27,6.32,6.39,0.07,1.08,-0.05,0.19,-0.85,3.00,Quantitive,0.01,All,Total


## Summarizing - ID in cities with liveness 24/7 enabled

## Summarizing - Triggers on liveness checking process (How it impact, how many committed incidents, scale of affected)

### Affectin on cities / share of triggered users

In [345]:
# df_effect_triggers = read_bq("""
#        WITH banhammer AS (SELECT user_id,
#                             city_id,
#                             country_id,
#                             DATE(created_at)                                             AS event_dt_part,
#                             ARRAY_TO_STRING(ARRAY_AGG(CAST(config_id AS string)), ', ')  AS config_id,
#                             ARRAY_TO_STRING(ARRAY_AGG(DISTINCT name), ', ')              AS name,
#                             ARRAY_TO_STRING(ARRAY_AGG(CAST(created_at AS string)), ', ') AS created_at,
#                             MAX(created_at)                                              AS last_created_at
#                      FROM (SELECT DISTINCT t1.user_id,
#                                           t1.city_id,
#                                           t1.country_id,
#                                           t1.config_id,
#                                           t1.created_at,
#                                           t2.name,
#                                           t2.city_ids,
#                             FROM indriver-e6e40.ods_banhammer.user_log_global t1
#                                    JOIN indriver-e6e40.ods_banhammer.actions_configs_global t2
#                                           ON t1.config_id = t2.id
#                             WHERE t1.created_at >= '2024-12-01'
#                             AND t1.config_id IN (10627, 10624, 10625, 10626))
#                      GROUP BY 1, 2, 3, 4),
#        liveness AS (SELECT user_id,
#                             os_name,
#                             event_dt_part,
#                             city_id,
#                             country_id,
#                             city_name,
#                             country_name,
#                             fulfilled_flow,
#                             MIN(IF(name = 'client.verification_start.show', client_time,
#                                    NULL)) AS banner_show,
#                             MIN(IF(name = 'client.verification_start.click', client_time,
#                                    NULL)) AS banner_click,
#                             MIN(IF(name = 'client.verification_flow_result_status.show' AND
#                                    LOWER(status) = 'approve',
#                                    client_time,
#                                    NULL)) AS status_result
#                      FROM (SELECT user_id,
#                                    name,
#                                    os_name,
#                                    event_dt_part,
#                                    TIMESTAMP_MILLIS(client_time)                                           AS client_time,
#                                    t1.city_id,
#                                    t2.city_name,
#                                    t2.country_id,
#                                    t2.country_name,
#                                    IF(LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
#                                           OVER (PARTITION BY user_id, event_dt_part ORDER BY client_time) IS NULL,
#                                    JSON_EXTRACT_SCALAR(payload, '$.verification_flow'),
#                                    LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
#                                           OVER (PARTITION BY user_id, event_dt_part ORDER BY client_time)) AS fulfilled_flow,
#                                    JSON_EXTRACT_SCALAR(payload, '$.status')                                AS status
#                             FROM indriver-e6e40.emart.swrve_event t1
#                                    JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
#                                           ON
#                                                  t1.city_id = t2.city_id
#                             WHERE 1 = 1
#                             AND name IN (
#                                           'client.verification_start.show',
#                                           'client.verification_start.click',
#                                           'client.verification_flow_result_status.show'
#                                    )
#                             AND event_dt_part >= '2024-12-01')
#                      WHERE fulfilled_flow = 'liveness'
#                      GROUP BY 1, 2, 3, 4, 5, 6, 7, 8),
#        orders AS (SELECT t1.city_id,
#                             city_name,
#                             metric_date_utc,
#                             COUNT(user_id) AS total_users
#                      FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
#                             JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
#                                    ON t1.city_id = t2.city_id
#                      WHERE metric_date_utc >= '2024-12-01'
#                      AND user_type = 'pass'
#                      GROUP BY 1, 2, 3),
#        joined AS
#               (SELECT t1.*,
#                      loc.city_name,
#                      loc.country_name,
#                      t2.user_id       AS live_user_id,
#                      t2.os_name,
#                      t2.event_dt_part AS live_event_dt_part,
#                      t2.city_id       AS live_city_id,
#                      t2.country_id    AS live_country_id,
#                      t2.fulfilled_flow,
#                      t2.banner_show,
#                      t2.banner_click,
#                      t2.status_result
#               FROM banhammer t1
#                      JOIN indriver-e6e40.heap.vw_macroregion_mapping loc
#                             ON
#                                    t1.city_id = loc.city_id
#                      LEFT JOIN liveness t2
#                                    ON t1.user_id = t2.user_id
#                                    AND banner_show >= last_created_at
#               WHERE os_name IN ('android', 'ios')
#               QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.user_id ORDER BY status_result) = 1)
#        SELECT t1.city_id,
#               t1.city_name,
#               t1.country_id,
#               t1.country_name,
#               SUM(t1.bh_triggered_users)                                         AS triggered_users,
#               SUM(t1.approved)                                                   AS approved,
#               SUM(t2.total_users)                                                AS total_users,
#               ROUND((SUM(t1.bh_triggered_users) / SUM(t2.total_users)) * 100, 2) AS impact
#        --        t1.city_id,
#        --        t1.city_name,
#        --        t1.country_id,
#        --        t1.country_name,
#        --        t1.event_dt_part,
#        --        t1.bh_triggered_users,
#        --        t1.banner_show,
#        --        t1.banner_click,
#        --        t1.approved,
#        --        t2.total_users,
#        --        ROUND((t1.bh_triggered_users / t2.total_users) * 100, 2) AS impact
#        FROM (SELECT city_id,
#               city_name,
#               country_id,
#               country_name,
#               event_dt_part,
#               COUNT(user_id)       AS bh_triggered_users,
#               COUNT(banner_show)   AS banner_show,
#               COUNT(banner_click)  AS banner_click,
#               COUNT(status_result) AS approved
#        FROM joined
#        GROUP BY 1, 2, 3, 4, 5) t1
#               JOIN orders t2
#                      ON t1.event_dt_part = t2.metric_date_utc
#                      AND t1.city_id = t2.city_id
#        GROUP BY 1, 2, 3, 4
# """)

df_effect_triggers_dr = read_bq("""
       WITH banhammer AS (SELECT user_id,
                            city_id,
                            country_id,
                            DATE(created_at)                                             AS event_dt_part_bh,
                            ARRAY_TO_STRING(ARRAY_AGG(CAST(config_id AS string)), ', ')  AS config_id,
                            ARRAY_TO_STRING(ARRAY_AGG(DISTINCT name), ', ')              AS name,
                            ARRAY_TO_STRING(ARRAY_AGG(CAST(created_at AS string)), ', ') AS created_at,
                            MAX(created_at)                                              AS last_created_at
                     FROM (SELECT DISTINCT t1.user_id,
                                          t1.city_id,
                                          t1.country_id,
                                          t1.config_id,
                                          t1.created_at,
                                          t2.name,
                                          t2.city_ids,
                            FROM indriver-e6e40.ods_banhammer.user_log_global t1
                                   JOIN indriver-e6e40.ods_banhammer.actions_configs_global t2
                                          ON t1.config_id = t2.id
                            WHERE t1.created_at >= '2025-01-20'
                            AND t1.config_id IN (10627, 10624, 10625, 10626))
                     GROUP BY 1, 2, 3, 4),
       liveness AS (SELECT user_id,
                            os_name,
                            event_dt_part,
                            city_id,
                            country_id,
                            city_name,
                            country_name,
                            fulfilled_flow,
                            MIN(IF(name = 'client.verification_start.show', client_time,
                                   NULL)) AS banner_show,
                            MIN(IF(name = 'client.verification_start.click', client_time,
                                   NULL)) AS banner_click,
                            MIN(IF(name = 'client.verification_flow_result_status.show' AND
                                   LOWER(status) = 'approve',
                                   client_time,
                                   NULL)) AS status_result
                     FROM (SELECT user_id,
                                   name,
                                   os_name,
                                   event_dt_part,
                                   TIMESTAMP_MILLIS(client_time)                                           AS client_time,
                                   t1.city_id,
                                   t2.city_name,
                                   t2.country_id,
                                   t2.country_name,
                                   IF(LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                                          OVER (PARTITION BY user_id, event_dt_part ORDER BY client_time) IS NULL,
                                   JSON_EXTRACT_SCALAR(payload, '$.verification_flow'),
                                   LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                                          OVER (PARTITION BY user_id, event_dt_part ORDER BY client_time)) AS fulfilled_flow,
                                   JSON_EXTRACT_SCALAR(payload, '$.status')                                AS status
                            FROM indriver-e6e40.ods_event_tracker.event t1
                                   JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                          ON
                                                 t1.city_id = t2.city_id
                            WHERE 1 = 1
                            AND name IN (
                                          'client.verification_start.show',
                                          'client.verification_start.click',
                                          'client.verification_flow_result_status.show'
                                   )
                            AND event_dt_part >= '2025-01-20')
                     WHERE fulfilled_flow = 'liveness'
                     GROUP BY 1, 2, 3, 4, 5, 6, 7, 8),
       orders AS (SELECT t1.city_id,
                            city_name,
                            metric_date_utc,
                            COUNT(DISTINCT user_id) AS total_users
                     FROM indriver-bi.incity.tbl_incity_growth_metrics_detail t1
                            JOIN `indriver-bi.heap.vw_macroregion_mapping` t2
                                   ON t1.city_id = t2.city_id
                     WHERE metric_date_utc >= '2025-01-20'
                     AND user_type = 'pass'
                     GROUP BY 1, 2, 3),
       rides AS (SELECT order_uuid,
                            user_id    AS pass_id,
                            driver_id,
                            city_id    AS order_city_id,
                            country_id AS order_country_id,
                            status_order,
                            order_timestamp,
                            at_pickup_dttm,
                            departed_pickup_dttm,
                            at_destination_dttm,
                            departed_destination_dttm,
                            driveraccept_timestamp,
                            driverarrived_timestamp,
                            driverstarttheride_timestamp,
                            driverdone_timestamp,
                            clientdone_timestamp,
                            clientcancel_timestamp,
                            drivercancel_timestamp,
                            user_reg_date,
                            driver_reg_date,
                            stage,
                            created_date_order_part,
                            duration_in_seconds
                     FROM indriver-e6e40.imart.incity_detail_new_order
                     WHERE created_date_order_part >= '2025-01-20'
                     AND status_order = 'RIDE_STATUS_DONE'
                     AND driverstarttheride_timestamp IS NOT NULL
                     AND (clientcancel_timestamp IS NULL
                     AND drivercancel_timestamp IS NULL)),
       joined AS
              (SELECT t1.*,
                     loc.city_name,
                     loc.country_name,
                     t2.user_id       AS live_user_id,
                     t2.os_name,
                     t2.event_dt_part AS live_event_dt_part,
                     t2.city_id       AS live_city_id,
                     t2.country_id    AS live_country_id,
                     t2.fulfilled_flow,
                     t2.banner_show,
                     t2.banner_click,
                     t2.status_result,
                     t3.*
              FROM banhammer t1
                     JOIN indriver-e6e40.heap.vw_macroregion_mapping loc
                            ON
                                   t1.city_id = loc.city_id
                     LEFT JOIN liveness t2
                                   ON t1.user_id = t2.user_id
                                   AND banner_show >= last_created_at
                     LEFT JOIN rides t3
                                   ON t2.user_id = t3.pass_id
                                   AND order_timestamp BETWEEN t2.status_result AND (t2.status_result + INTERVAL 1 DAY)
              QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.user_id ORDER BY order_timestamp) = 1)
       SELECT t1.*,
              t2.total_users
       FROM (SELECT t1.city_id,
              t1.city_name,
              t1.country_name,
              event_dt_part_bh,
              COUNT(user_id)       AS triggered,
              COUNT(banner_show)   AS banner_show,
              COUNT(status_result) AS result,
              COUNT(order_uuid)    AS orders
       FROM joined t1
       GROUP BY 1, 2, 3, 4) t1
              LEFT JOIN orders t2
                     ON t1.city_id = t2.city_id
                            AND t1.event_dt_part_bh = t2.metric_date_utc
""")
df_effect_triggers_dr['event_dt_part_bh'] = pd.to_datetime(df_effect_triggers_dr['event_dt_part_bh'])
df_effect_triggers_dr.head()

,city_id,city_name,country_name,event_dt_part_bh,triggered,banner_show,result,orders,total_users
0,4196,Cucuta,Colombia,2025-02-03,646,142,125,83,14200
1,4196,Cucuta,Colombia,2025-01-27,622,172,142,107,13144
2,4257,Trujillo,Peru,2025-01-23,887,632,553,447,49009
3,4261,Concepción,Chile,2025-02-09,247,125,110,64,4154
4,4263,Antofagasta,Chile,2025-01-25,534,405,352,299,18729


In [484]:
df_agg = df_effect_triggers_dr.groupby(['city_name', 'country_name'], as_index=False)[['triggered', 'banner_show', 'result', 'orders', 'total_users']].sum()

df_agg['effect'] = df_agg['triggered'] / df_agg['total_users'] * 100
df_agg['cr_to_banner_show'] = df_agg['banner_show'] / df_agg['triggered'] * 100
df_agg['cr_to_approved_triggered'] = df_agg['result'] / df_agg['banner_show'] * 100
df_agg['cr_to_ride_banner_show'] = df_agg['orders'] / df_agg['banner_show'] * 100
df_agg['cr_to_ride_triggered'] = df_agg['orders'] / df_agg['triggered'] * 100
df_agg['cr_to_ride_approved'] = df_agg['orders'] / df_agg['result'] * 100
df_agg['period'] = '20 Jan - 18 Feb'

df_agg = df_agg.sort_values('triggered', ascending=False)

df_agg_with_perc = df_agg.copy()
columns = ['effect', 'cr_to_banner_show', 'cr_to_approved_triggered', 'cr_to_ride_banner_show', 'cr_to_ride_triggered', 'cr_to_ride_approved']

for column in columns:
    df_agg_with_perc[column] = df_agg_with_perc[column].round(2).astype(str)

def perc_add(row):
    return row + '%'

df_agg_with_perc['effect'] = df_agg_with_perc['effect'].apply(perc_add)
df_agg_with_perc['cr_to_banner_show'] = df_agg_with_perc['cr_to_banner_show'].apply(perc_add)
df_agg_with_perc['cr_to_approved_triggered'] = df_agg_with_perc['cr_to_approved_triggered'].apply(perc_add)
df_agg_with_perc['cr_to_ride_banner_show'] = df_agg_with_perc['cr_to_ride_banner_show'].apply(perc_add)
df_agg_with_perc['cr_to_ride_triggered'] = df_agg_with_perc['cr_to_ride_triggered'].apply(perc_add)
df_agg_with_perc['cr_to_ride_approved'] = df_agg_with_perc['cr_to_ride_approved'].apply(perc_add)


df_agg_with_perc.head(3)

,city_name,country_name,triggered,banner_show,result,orders,total_users,effect,cr_to_banner_show,cr_to_approved_triggered,cr_to_ride_banner_show,cr_to_ride_triggered,cr_to_ride_approved,period
21,Lima,Peru,300389,99296,80652,48463,10913065,2.75%,33.06%,81.22%,48.81%,16.13%,60.09%,20 Jan - 18 Feb
23,Medellin,Colombia,128411,51948,42903,26990,4179923,3.07%,40.45%,82.59%,51.96%,21.02%,62.91%,20 Jan - 18 Feb
37,São Paulo,Brazil,77864,21990,18380,8412,1482566,5.25%,28.24%,83.58%,38.25%,10.8%,45.77%,20 Jan - 18 Feb


In [485]:
path = '/Users/renatyunison/Downloads/1 row of Metrics Table.xlsx'

df_csv_dr = pd.read_excel(path)

df_csv_dr = df_csv_dr[1:]
df_csv_dr = df_csv_dr.drop(columns=['Unnamed: 0', 'Unnamed: 2', 'Total'])
df_csv_dr = pd.melt(df_csv_dr, id_vars=['Unnamed: 1']).rename(columns={'Unnamed: 1':'city_name', 'variable':'date'})
df_csv_dr['date'] = pd.to_datetime(df_csv_dr['date'])
df_csv_dr['value'] = df_csv_dr['value'].astype('float')

list_of_cities = list(df_effect_triggers_dr['city_name'].unique())
df_csv_dr_agg = df_csv_dr[(df_csv_dr['city_name'].isin(list_of_cities))].query("date >= '2025-01-01' and date <= '2025-02-17'").groupby(['city_name'], as_index=False)['value'].mean()
df_csv_dr_agg['value'] = df_csv_dr_agg['value']*100

df_csv_dr_agg.head()

,city_name,value
0,Antofagasta,67.37
1,Arica,68.55
2,Belo Horizonte,28.89
3,Belém,35.08
4,Brasília,45.65


In [486]:
df_total_dr = df_agg.merge(df_csv_dr_agg, left_on='city_name', right_on='city_name', how='inner')
df_total_dr['dr_delta_banner_show_cr'] = df_total_dr['cr_to_ride_banner_show'] - df_total_dr['value']

columns = ['effect', 'cr_to_banner_show', 'cr_to_approved_triggered', 'cr_to_ride_banner_show', 'cr_to_ride_triggered', 'cr_to_ride_approved', 'value', 'dr_delta_banner_show_cr']

for column in columns:
    df_total_dr[column] = df_total_dr[column].round(2).astype(str)

def perc_add(row):
    return row + '%'

df_total_dr['effect'] = df_total_dr['effect'].apply(perc_add)
df_total_dr['cr_to_banner_show'] = df_total_dr['cr_to_banner_show'].apply(perc_add)
df_total_dr['cr_to_approved_triggered'] = df_total_dr['cr_to_approved_triggered'].apply(perc_add)
df_total_dr['cr_to_ride_triggered'] = df_total_dr['cr_to_ride_triggered'].apply(perc_add)
df_total_dr['cr_to_ride_approved'] = df_total_dr['cr_to_ride_approved'].apply(perc_add)
df_total_dr['cr_to_ride_banner_show'] = df_total_dr['cr_to_ride_banner_show'].apply(perc_add)
df_total_dr['value'] = df_total_dr['value'].apply(perc_add)
df_total_dr['dr_delta_banner_show_cr'] = df_total_dr['dr_delta_banner_show_cr'].apply(perc_add)

df_total_dr

,city_name,country_name,triggered,banner_show,result,orders,total_users,effect,cr_to_banner_show,cr_to_approved_triggered,cr_to_ride_banner_show,cr_to_ride_triggered,cr_to_ride_approved,period,value,dr_delta_banner_show_cr
0,Lima,Peru,300389,99296,80652,48463,10913065,2.75%,33.06%,81.22%,48.81%,16.13%,60.09%,20 Jan - 18 Feb,54.34%,-5.53%
1,Medellin,Colombia,128411,51948,42903,26990,4179923,3.07%,40.45%,82.59%,51.96%,21.02%,62.91%,20 Jan - 18 Feb,55.42%,-3.47%
2,São Paulo,Brazil,77864,21990,18380,8412,1482566,5.25%,28.24%,83.58%,38.25%,10.8%,45.77%,20 Jan - 18 Feb,41.46%,-3.21%
3,Cape Town,South Africa,73290,26641,20685,11566,1556484,4.71%,36.35%,77.64%,43.41%,15.78%,55.91%,20 Jan - 18 Feb,53.13%,-9.71%
4,Manaus,Brazil,39154,39,36,11,901249,4.34%,0.1%,92.31%,28.21%,0.03%,30.56%,20 Jan - 18 Feb,42.37%,-14.16%
5,Guayaquil,Ecuador,37856,9217,8099,4781,952473,3.97%,24.35%,87.87%,51.87%,12.63%,59.03%,20 Jan - 18 Feb,56.79%,-4.91%
6,Johannesburg,South Africa,33502,13726,11171,5098,336381,9.96%,40.97%,81.39%,37.14%,15.22%,45.64%,20 Jan - 18 Feb,43.48%,-6.34%
7,Recife,Brazil,32040,7504,6118,2865,787753,4.07%,23.42%,81.53%,38.18%,8.94%,46.83%,20 Jan - 18 Feb,39.64%,-1.46%
8,Santiago,Chile,28637,15197,13143,8002,535096,5.35%,53.07%,86.48%,52.66%,27.94%,60.88%,20 Jan - 18 Feb,52.16%,0.49%
9,Trujillo,Peru,28295,19018,16519,12938,1636785,1.73%,67.21%,86.86%,68.03%,45.73%,78.32%,20 Jan - 18 Feb,67.65%,0.38%


### Incidents in context of the triggers

In [290]:
df_incidents = read_bq("""
       WITH bh AS (SELECT user_id,
                     city_id,
                     city_name,
                     country_id,
                     country_name,
                     DATE(created_at)                                  AS event_dt_part,
                     STRING_AGG(DISTINCT name, ', ' ORDER BY name ASC) AS name,
                     MIN(created_at)                                   AS first_created_at,
                     MAX(created_at)                                   AS last_created_at
              FROM (SELECT DISTINCT t1.user_id,
                                   t1.city_id,
                                   v.city_name,
                                   v.country_name,
                                   t1.country_id,
                                   t1.config_id,
                                   t1.created_at,
                                   t2.name,
                                   t2.city_ids,
                     FROM indriver-e6e40.ods_banhammer.user_log_global t1
                            JOIN indriver-e6e40.ods_banhammer.actions_configs_global t2
                                   ON t1.config_id = t2.id
                            JOIN indriver-e6e40.heap.vw_macroregion_mapping v
                                   ON
                                          t1.city_id = v.city_id
                     WHERE t1.created_at >= '2025-01-20'
                     AND t1.config_id IN (10627, 10624, 10625, 10626))
              GROUP BY 1, 2, 3, 4, 5, 6),
       incidents AS (SELECT driver_id AS aggressor_id,
                            'driver'  AS mode,
                            redmine_id,
                            incident_level,
                            incident_date,
                            city_id,
                            city_name,
                     FROM indriver-bi.safety.vw_safety_incidents_detail
                     WHERE incident_date >= '2025-01-20'
                            AND information_status = 'Confirmed'
                            AND aggressor = 'Driver'
                            AND city_id IN
                            (4516, 4257, 4228, 4549, 4264, 5536, 4142, 5528, 4200, 5535, 4231, 4153, 4373, 5600, 4269,
                            4226,
                            4397, 4755, 5589,
                            4517, 4242, 4524, 4199, 4229, 5368, 4374, 4299, 4271, 4385, 798, 4154, 4376, 4196, 4266, 5573,
                            4261,
                            4300, 4559,
                            5568, 4530, 4545, 4540, 4267, 4263)
                     UNION ALL
                     SELECT pass_id AS aggressor_id,
                            'pass'  AS mode,
                            redmine_id,
                            incident_level,
                            incident_date,
                            city_id,
                            city_name,
                     FROM indriver-bi.safety.vw_safety_incidents_detail
                     WHERE incident_date >= '2025-01-20'
                            AND city_id IN
                            (4516, 4257, 4228, 4549, 4264, 5536, 4142, 5528, 4200, 5535, 4231, 4153, 4373, 5600, 4269,
                            4226,
                            4397, 4755, 5589,
                            4517, 4242, 4524, 4199, 4229, 5368, 4374, 4299, 4271, 4385, 798, 4154, 4376, 4196, 4266, 5573,
                            4261,
                            4300, 4559,
                            5568, 4530, 4545, 4540, 4267, 4263)
                            AND information_status = 'Confirmed'
                            AND aggressor = 'Passenger')

       SELECT t1.country_id,
              t1.country_name,
              t1.name,
              COUNT(user_id)    AS triggered_all,
              COUNT(redmine_id) AS incidents
       FROM bh t1
              LEFT JOIN incidents t2
                     ON t1.user_id = t2.aggressor_id
       GROUP BY 1, 2, 3
""")

df_incidents['share'] = df_incidents['incidents'] / df_incidents['triggered_all']*100000
df_incidents['period'] = '20 Jan - 18 Feb'


df_incidents.head()

,country_id,country_name,name,triggered_all,incidents,share
0,54,Jamaica,change_of_device_liveness_trigger_24_7_checks,8170,3,36.72
1,10,South Africa,"change_of_avatar_liveness_trigger_24_7_checks, change_of_device_liveness_trigger_24_7_checks",135,0,0.00
2,43,Ecuador,suspicious_poor_rating_liveness_trigger_24_7_checks,767,4,521.51
3,11,Brazil,"change_of_avatar_liveness_trigger_24_7_checks, change_of_device_liveness_trigger_24_7_checks",229,1,436.68
4,24,Peru,"change_of_device_liveness_trigger_24_7_checks, suspicious_poor_rating_liveness_trigger_24_7_checks",33,0,0.00


In [298]:
df_incidents.nlargest(10, 'incidents')

,country_id,country_name,name,triggered_all,incidents,share,period
37,11,Brazil,change_of_device_liveness_trigger_24_7_checks,266780,176,65.97,20 Jan - 18 Feb
88,24,Peru,change_of_device_liveness_trigger_24_7_checks,252350,91,36.06,20 Jan - 18 Feb
76,10,South Africa,change_of_device_liveness_trigger_24_7_checks,87322,43,49.24,20 Jan - 18 Feb
47,25,Chile,suspicious_poor_rating_liveness_trigger_24_7_checks,2912,42,"1,442.31",20 Jan - 18 Feb
56,25,Chile,change_of_device_liveness_trigger_24_7_checks,96189,41,42.62,20 Jan - 18 Feb
42,11,Brazil,suspicious_poor_rating_liveness_trigger_24_7_checks,6031,40,663.24,20 Jan - 18 Feb
21,11,Brazil,change_of_name_liveness_trigger_24_7_checks,6149,34,552.94,20 Jan - 18 Feb
72,25,Chile,change_of_name_liveness_trigger_24_7_checks,11669,28,239.95,20 Jan - 18 Feb
14,22,Colombia,change_of_device_liveness_trigger_24_7_checks,133815,26,19.43,20 Jan - 18 Feb
31,24,Peru,change_of_name_liveness_trigger_24_7_checks,47436,23,48.49,20 Jan - 18 Feb


In [292]:
df_incidents['incidents'].sum()

751

In [293]:
df_incidents_all = read_bq("""
    WITH bh AS (SELECT user_id,
                   city_id,
                   city_name,
                   country_id,
                   country_name,
                   DATE(created_at)                                  AS event_dt_part,
                   STRING_AGG(DISTINCT name, ', ' ORDER BY name ASC) AS name,
                   MIN(created_at)                                   AS first_created_at,
                   MAX(created_at)                                   AS last_created_at
            FROM (SELECT DISTINCT t1.user_id,
                                  t1.city_id,
                                  v.city_name,
                                  v.country_name,
                                  t1.country_id,
                                  t1.config_id,
                                  t1.created_at,
                                  t2.name,
                                  t2.city_ids,
                  FROM indriver-e6e40.ods_banhammer.user_log_global t1
                           JOIN indriver-e6e40.ods_banhammer.actions_configs_global t2
                                ON t1.config_id = t2.id
                           JOIN indriver-e6e40.heap.vw_macroregion_mapping v
                                ON
                                    t1.city_id = v.city_id
                  WHERE t1.created_at >= '2025-01-20'
                    AND t1.config_id IN (10627, 10624, 10625, 10626))
            GROUP BY 1, 2, 3, 4, 5, 6),
     incidents AS (SELECT driver_id AS aggressor_id,
                          'driver'  AS mode,
                          redmine_id,
                          incident_level,
                          incident_date,
                          city_id,
                          city_name,
                          country_name
                   FROM indriver-bi.safety.vw_safety_incidents_detail
                   WHERE incident_date >= '2025-01-20'
                     AND information_status = 'Confirmed'
                     AND aggressor = 'Driver'
                     AND city_id IN
                         (4516, 4257, 4228, 4549, 4264, 5536, 4142, 5528, 4200, 5535, 4231, 4153, 4373, 5600, 4269,
                          4226,
                          4397, 4755, 5589,
                          4517, 4242, 4524, 4199, 4229, 5368, 4374, 4299, 4271, 4385, 798, 4154, 4376, 4196, 4266, 5573,
                          4261,
                          4300, 4559,
                          5568, 4530, 4545, 4540, 4267, 4263)
                   UNION ALL
                   SELECT pass_id AS aggressor_id,
                          'pass'  AS mode,
                          redmine_id,
                          incident_level,
                          incident_date,
                          city_id,
                          city_name,
                          country_name
                   FROM indriver-bi.safety.vw_safety_incidents_detail
                   WHERE incident_date >= '2025-01-20'
                     AND city_id IN
                         (4516, 4257, 4228, 4549, 4264, 5536, 4142, 5528, 4200, 5535, 4231, 4153, 4373, 5600, 4269,
                          4226,
                          4397, 4755, 5589,
                          4517, 4242, 4524, 4199, 4229, 5368, 4374, 4299, 4271, 4385, 798, 4154, 4376, 4196, 4266, 5573,
                          4261,
                          4300, 4559,
                          5568, 4530, 4545, 4540, 4267, 4263)
                     AND information_status = 'Confirmed'
                     AND aggressor = 'Passenger')
    SELECT t1.country_name,
        t1.incident_level,
        '20 Jan - 18 Feb'                                    AS period,
        COUNT(redmine_id)                                    AS incidents,
        COUNT(user_id)                                       AS triggered,
        SAFE_DIVIDE(COUNT(user_id), COUNT(redmine_id)) * 100 AS share_of_detected
    FROM incidents t1
            LEFT JOIN bh t2
                    ON t1.aggressor_id = t2.user_id
    GROUP BY 1, 2, 3
""")

df_incidents_all.head()

,country_name,incident_level,period,incidents,triggered,share_of_detected
0,Colombia,Red,20 Jan - 18 Feb,15,2,13.33
1,Chile,Yellow,20 Jan - 18 Feb,155,83,53.55
2,South Africa,Yellow,20 Jan - 18 Feb,57,16,28.07
3,South Africa,Green,20 Jan - 18 Feb,184,53,28.80
4,Jamaica,Yellow,20 Jan - 18 Feb,7,3,42.86


In [302]:
df_incidents_all.sort_values(['country_name', 'incident_level'], ascending=True)

,country_name,incident_level,period,incidents,triggered,share_of_detected
11,Brazil,Green,20 Jan - 18 Feb,383,82,21.41
16,Brazil,Red,20 Jan - 18 Feb,77,24,31.17
23,Brazil,Yellow,20 Jan - 18 Feb,709,161,22.71
22,Chile,Green,20 Jan - 18 Feb,107,30,28.04
24,Chile,Red,20 Jan - 18 Feb,35,16,45.71
1,Chile,Yellow,20 Jan - 18 Feb,155,83,53.55
5,Colombia,Green,20 Jan - 18 Feb,73,10,13.70
0,Colombia,Red,20 Jan - 18 Feb,15,2,13.33
17,Colombia,Yellow,20 Jan - 18 Feb,188,43,22.87
13,Dominican Republic,Green,20 Jan - 18 Feb,4,0,0.00


### Inconsistency in the pictures by sumsbub

In [269]:
df_response_sumsub = pd.read_csv('/Users/renatyunison/Downloads/Rejected 2024 - 2025.csv')

def checker(row):
    if row['externalUserId'].find('level') >= 0:
        return 1
    else:
        return 0

df_response_sumsub['checker'] = df_response_sumsub.apply(checker, axis=1)
df_response_sumsub = df_response_sumsub.drop(index=df_response_sumsub[df_response_sumsub['checker']==1].index)
df_response_sumsub['externalUserId'] = df_response_sumsub['externalUserId'].astype('int64')

df_response_sumsub.head()

,applicantId,externalUserId,reviewDate,rejectLabels,buttonIds
0,6774b30ff0b0e206918d5cb4,273812956,2025-01-01 03:16:47.618,['FORGERY'],"['fake_fakeSelfie','fake']"
1,6774b373b92aec251e8c451f,270035138,2025-01-01 03:24:29.764,['FORGERY'],"['fake_fakeSelfie','fake']"
2,6774b39226092808d5fae9dc,274018158,2025-01-01 03:51:08.691,['FORGERY'],"['fake_fakeSelfie','fake']"
3,6774b8cb10b5c75bc4c7e020,274025825,2025-01-01 03:41:45.421,['FORGERY'],"['fake_fakeSelfie','fake']"
4,6774b8d2fecd2018b448cbbb,274022234,2025-01-01 03:40:52.377,['FORGERY'],"['fake_fakeSelfie','fake']"


In [303]:
df_all_bh = read_bq("""
SELECT user_id,
       city_id,
       country_id,
       DATE(created_at)                                  AS event_dt_part,
       STRING_AGG(CAST(config_id AS STRING))             AS config_id,
       STRING_AGG(DISTINCT name, ', ' ORDER BY name ASC) AS name,
       STRING_AGG(CAST(created_at AS string))            AS created_at,
       MAX(created_at)                                   AS last_created_at
FROM (SELECT DISTINCT t1.user_id,
                      t1.city_id,
                      t1.country_id,
                      t1.config_id,
                      t1.created_at,
                      t2.name,
                      t2.city_ids,
      FROM indriver-e6e40.ods_banhammer.user_log_global t1
               JOIN indriver-e6e40.ods_banhammer.actions_configs_global t2
                    ON t1.config_id = t2.id
      WHERE t1.created_at >= '2025-01-20'
        AND t1.config_id IN (10627, 10624, 10625, 10626))
GROUP BY 1, 2, 3, 4
""")

df_all_bh.head()

In [333]:
df_rejected = df_all_bh.merge(df_response_sumsub, left_on='user_id', right_on='externalUserId', how='left')

In [335]:
df_rejected.groupby(['rejectLabels', 'buttonIds'], as_index=False)['user_id'].count()

,rejectLabels,buttonIds,user_id
0,"['AGE_REQUIREMENT_MISMATCH','REGULATIONS_VIOLATIONS']","['regulationsViolations','regulationsViolations_age']",6
1,['BLOCKLIST'],"['fraudulentPatterns','fraudulentPatterns_blocklist']",1
2,['DB_DATA_NOT_FOUND'],"['dbNetReject','dbNetReject_dataNotFound']",23
3,['FORGERY'],"['dbNetRetry','fake_fakeSelfie','dbNetRetry_dataNotFound','fake']",1
4,['FORGERY'],"['fake_fakeLiveness','fake']",81
5,['FORGERY'],"['fake_fakeSelfie','fake']",358
6,['FORGERY'],"['fake_fakeSelfie','fake_fakeLiveness','fake']",5
7,['FORGERY'],"['fraudulentPatterns','fraudulentPatterns_forcedVerification']",1
8,['FRAUDULENT_LIVENESS'],"['selfieIssues_livenessBypass','selfieIssues']",2
9,['INCONSISTENT_PROFILE'],['differentDocs'],1


### Fixing the triggers

In [488]:
df_bh_settings = read_bq("""
WITH cte AS (SELECT DISTINCT t1.user_id,
                             t1.city_id,
                             t1.country_id,
                             t1.config_id,
                             t1.created_at,
                             t2.name,
                             t2.city_ids,
             FROM indriver-e6e40.ods_banhammer.user_log_global t1
                      JOIN indriver-e6e40.ods_banhammer.actions_configs_global t2
                           ON t1.config_id = t2.id
             WHERE t1.created_at >= '2025-01-20'
               AND t1.config_id IN (10627, 10624, 10625, 10626))
SELECT name,
       city_id,
       DATE(created_at) AS dt,
       COUNT(user_id) as users
FROM cte
GROUP BY 1, 2, 3
""")

df_bh_settings.head()

,name,city_id,dt,users
0,change_of_name_liveness_trigger_24_7_checks,4199,2025-02-14,4591
1,change_of_name_liveness_trigger_24_7_checks,4200,2025-02-14,264
2,change_of_name_liveness_trigger_24_7_checks,4271,2025-02-14,120
3,change_of_name_liveness_trigger_24_7_checks,5536,2025-02-14,59
4,change_of_avatar_liveness_trigger_24_7_checks,4142,2025-02-14,161


In [491]:
px.line(df_bh_settings.groupby(['name', 'dt'], as_index=False)['users'].sum(),
x='dt',
y='users',
color='name')

In [489]:
df_bh_settings.pivot_table(columns='name', index='city_id', values='users', aggfunc='sum')

name,change_of_avatar_liveness_trigger_24_7_checks,change_of_device_liveness_trigger_24_7_checks,change_of_name_liveness_trigger_24_7_checks,suspicious_poor_rating_liveness_trigger_24_7_checks
city_id,,,,
798,2309,8277,4630,56
4142,5689,77068,4552,2418
4153,1041,14726,1040,175
4154,1111,7543,799,30
4196,3521,14572,5117,182
4199,83428,244383,129990,10305
4200,10603,23062,10071,786
4226,4029,8055,4864,510
4228,1684,7672,1101,318
